In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
from google.colab.patches import cv2_imshow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model 
from sklearn.utils.class_weight import compute_class_weight 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout,  GlobalAveragePooling2D
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

Setting up on-demand loading

In [ ]:
root = '/content/drive/MyDrive/Final Year Project/Data'#training data comes from multiple sources and they are stored in the 'data' folder

In [ ]:
input_dirs = [root+'/G1020/Images_Cropped/img', root+'/ORIGA/Images_Cropped']
labels = [root+'/G1020/G1020.csv', root+'/ORIGA/OrigaList.csv']

In [ ]:
input_files = []
for dir in input_dirs:
    input_files.extend([os.path.join(dir, f) for f in sorted(os.listdir(dir))])

df1 = pd.read_csv(labels[0])
df2 = pd.read_csv(labels[1])

col1 = df1.iloc[:, 1].tolist()
col2 = df2.iloc[:, 4].tolist()
new_col = col1 + col2

data = pd.DataFrame({'Input': input_files, 'Label': new_col})
data['Label'] = data['Label'].astype(str)

In [ ]:
class_weights = compute_class_weight('balanced', classes=['0', '1'], y=data['Label'])

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='nearest')

generator = datagen.flow_from_dataframe(
    dataframe=data,
    x_col='Input',
    y_col='Label',
    target_size=(256, 256), 
    batch_size=32,
    shuffle=True, 
    class_mode='binary',
    class_weight={'0': class_weights[0], '1': class_weights[1]})

Found 1670 validated image filenames belonging to 2 classes.


In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

for layer in base_model.layers:
    layer.trainable = False

x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(generator, steps_per_epoch=len(data) // 32, epochs=90)

In [ ]:
keras.models.save_model(model, '/content/drive/MyDrive/Final Year Project/Models/ImageClassifierCNN.h5')

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/Final Year Project/Models/ImageClassifierCNN.h5')

In [ ]:
model.summary()

In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

for layer in base_model.layers:
    layer.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=x)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

87910968/87910968 [==============================] - 0s 0us/step


In [ ]:
model.fit(generator, steps_per_epoch=len(data) // 32, epochs=90)

In [ ]:
keras.models.save_model(model, '/content/drive/MyDrive/Final Year Project/Models/ImageClassifierInception.h5')

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/Final Year Project/Models/ImageClassifierInception.h5')

In [ ]:
model.summary()

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

for layer in base_model.layers:
    layer.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=x)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

94765736/94765736 [==============================] - 4s 0us/step


In [ ]:
model.fit(generator, steps_per_epoch=len(data) // 32, epochs=90)

In [ ]:
keras.models.save_model(model, '/content/drive/MyDrive/Final Year Project/Models/ImageClassifierResNet.h5')

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/Final Year Project/Models/ImageClassifierResNet.h5')

In [ ]:
model.summary()

In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

for layer in base_model.layers:
    layer.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=x)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

9406464/9406464 [==============================] - 1s 0us/step


In [ ]:
model.fit(generator, steps_per_epoch=len(data) // 32, epochs=90)

In [ ]:
keras.models.save_model(model, '/content/drive/MyDrive/Final Year Project/Models/ImageClassifierMobileNet.h5')

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/Final Year Project/Models/ImageClassifierMobileNet.h5')

In [ ]:
model.summary()

In [ ]:
image = cv2.imread(input_files[1])
# get the original height and width of the image
height, width = image.shape[:2]

# calculate the aspect ratio of the image
aspect_ratio = float(width) / float(height)

# determine which dimension will be the longer side and scale it down to 256 pixels
if height > width:
    new_height = 256
    new_width = int(new_height * aspect_ratio)
else:
    new_width = 256
    new_height = int(new_width / aspect_ratio)

# resize the image
resized_img = cv2.resize(image, (new_width, new_height))

# create a black background with a 256x256 shape
background = np.zeros((256, 256, 3), dtype=np.uint8)

# calculate the center coordinates for the resized image
x_offset = int((256 - new_width) / 2)
y_offset = int((256 - new_height) / 2)

# insert the resized image into the center of the black background
background[y_offset:y_offset+new_height, x_offset:x_offset+new_width] = resized_img
image = background

image = np.expand_dims(image, axis=0)

In [ ]:
pred = model.predict(image)

1/1 [==============================] - 1s 1s/step


In [ ]:
pred[0][0]

0.64166623

In [ ]:
pred[0][0]

0.0

In [ ]:
binary_pred = np.argmax(pred, axis=1)

In [ ]:
threshold = 0.5
binary_pred = int(pred[:, 0] >= threshold)


In [ ]:
binary_pred

1

In [ ]:
pred[:, 0] > threshold

array([ True])